In [1]:
# PYTHON CODE
# Module imports

from scipy import interpolate
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

from IPython.core.display import display, HTML

In [10]:
#DATA

def temperature_correction(input_temperature):
    #Dictionary of data
    phantom_v2 = {
        '14': {'16': '21.94', '18': '21.62','20': '21.44', '22': '21.28', '24': '21.26', '26': '21.31'},
        '13': {'16': '31.05', '18': '30.65','20': '30.40', '22': '30.27', '24': '30.25', '26': '30.31'},
        '12': {'16': '43.79', '18': '43.24','20': '42.89', '22': '42.72', '24': '42.70', '26': '42.80'},
        '11': {'16': '61.49', '18': '60.70','20': '60.21', '22': '59.97', '24': '60.00', '26': '60.17'},
        '10': {'16': '87.47', '18': '86.41','20': '85.75', '22': '85.03', '24': '85.01', '26': '85.28'},
        '9': {'16': '122.99', '18': '121.79','20': '121.08', '22': '120.80', '24': '120.90', '26': '121.34'},
        '8': {'16': '177.68', '18': '175.94','20': '174.95', '22': '174.59', '24': '174.78', '26': '175.48'},
        '7': {'16': '243.77', '18': '241.84','20': '240.86', '22': '240.75', '24': '241.31', '26': '242.45'},
        '6': {'16': '343.00', '18': '341.53','20': '341.58', '22': '342.58', '24': '344.23', '26': '346.67'},
        '5': {'16': '483.91', '18': '482.91','20': '484.97', '22': '486.92', '24': '490.24', '26': '494.55'},
        '4': {'16': '675.07', '18': '686.88','20': '690.08', '22': '695.01', '24': '701.06', '26': '709.48'},
        '3': {'16': '950.71', '18': '963.56','20': '987.27', '22': '1000.81', '24': '1015.70', '26': '1030.78'},
        '2': {'16': '1274.07', '18': '1317.71','20': '1330.16', '22': '1355.29', '24': '1367.79', '26': '1395.94'},
        '1': {'16': '1766.68', '18': '1830.34','20': '1883.97', '22': '1937.34', '24': '1987.50', '26': '2066.95'}
        };
    
    ##Get keys and values of dictionary to construct a 2D array##
    #Get dictionary keys as lists
    sphere = list(phantom_v2.keys());
    temperature = list(phantom_v2['1'].keys());

    #Define array (data2fit) to store data to be fitted
    data2fit = np.empty([len(temperature),len(sphere)+1]);
    #Fill data2fit array
    row = 0;
    for i in temperature:
        row = row + 1;
        data2fit[row-1,0] = int(i);
        for j in sphere:
            T1_value = float(phantom_v2[j][i]);
            data2fit[row-1,int(j)] = T1_value;
    
    ##Code for Temperature Correction: Interpolation##
    cs_estimatedT1_values = np.empty([len(input_temperature),len(sphere)]);
    cs_outDic_T1values = {};
    
    c_estimatedT1_values = np.empty([len(input_temperature),len(sphere)]);
    c_outDic_T1values = {};
    #Interpolations
    for k in range(len(sphere)):
        for l in range(len(input_temperature)):
            f_cubicSpline = interpolate.splrep(data2fit[:,0], data2fit[:,k+1]);
            #input_temperature2 = input_temperature[l];
            cs_estimatedT1_values[l,k] = interpolate.splev(input_temperature[l],f_cubicSpline);
            cs_outDic_T1values[l,k+1] = cs_estimatedT1_values[l,k];
        
            f_cubic = interpolate.interp1d(data2fit[:,0], data2fit[:,k+1], kind='cubic');
            c_estimatedT1_values[l,k] = f_cubic(input_temperature[l]);
            c_outDic_T1values[l,k+1] = c_estimatedT1_values[l,k];
    
    #Interpolation of data for Sphere No. 1 only
    #plt.plot(data2fit[:,0], data2fit[:,1], 'o')
    #plt.plot(data2fit[:,0], cs_estimatedT1_values[:,0], '--');
    #plt.plot(data2fit[:,0], c_estimatedT1_values[:,0], '-*');
    #plt.legend(['data', 'cubic-spline', 'cubic'], loc='best');
    #plt.title('Interpolation: Sphere No. 1');
    #plt.xlabel('Temperature (°C)');
    #plt.ylabel('T1 value (ms)')
    #plt.show()
    
# PLOTS
    init_notebook_mode(connected=True)
    cs_estimatedT1_values = np.transpose(cs_estimatedT1_values)
    c_estimatedT1_values = np.transpose(c_estimatedT1_values)
    original_data = np.transpose(data2fit)
    original_data = np.delete(original_data, 0, 0)
    
    data1 = [dict(
        visible = False,
        line=dict(color='orange'),
        x = temperature,
        y = abs(original_data[13-ii]),
        name = 'Data',
        text = 'Data)',
        hoverinfo = 'x+y+text') for ii in range(len(sphere))]

    data1[4]['visible'] = True
    
    data2 = [dict(
        visible = False,
        line=dict(color='royalblue'),
        x = input_temperature,
        y = abs(cs_estimatedT1_values[13-ii]),
        name = 'Cubic-Spline',
        text = 'Cubic-Spline',
        hoverinfo = 'x+y+text') for ii in range(len(sphere))]

    data2[4]['visible'] = True

    data3 = [dict(
        visible = False,
        line=dict(color='firebrick'),
        x = temperature,
        y = abs(c_estimatedT1_values[13-ii]),
        name = 'Cubic',
        text = 'Cubic)',
        hoverinfo = 'x+y+text') for ii in range(len(sphere))]

    data3[4]['visible'] = True

    data = data1 + data2 + data3

    steps = []
    for i in range(len(np.asarray(sphere))):
        step = dict(
            method = 'restyle',  
            args = ['visible', [False] * len(data2)],
            label = str(np.asarray(sphere)[i])
            )
        step['args'][1][i] = True # Toggle i'th trace to "visible"
        steps.append(step)

    sliders=[
        dict(
            x = 0,
            y = -0.09,
            active = 3,
            currentvalue = {"prefix": "Sphere No.: <b>"},
            pad = {"t": 50, "b": 10},
            steps = steps)]

    layout = go.Layout(
        width=580,
        height=450,
        margin=go.layout.Margin(
            l=80,
            r=40,
            b=60,
            t=10,
        ),
        annotations=[
            dict(
                x=0.5004254919715793,
                y=-0.18,
                showarrow=False,
                text='Temperature (°C)',
                font=dict(
                    family='Times New Roman',
                    size=22
                ),
                xref='paper',
                yref='paper'
            ),
            dict(
                x=-0.15,
                y=0.5,
                showarrow=False,
                text='Longitudinal Relaxation (T<sub>1</sub>)',
                font=dict(
                    family='Times New Roman',
                    size=22
                ),
                textangle=-90,
                xref='paper',
                yref='paper'
            ),
        ],
        xaxis=dict(
            autorange=False,
            range=[16, temperature[-1]],
            showgrid=False,
            linecolor='black',
            linewidth=2
        ),
        yaxis=dict(
            autorange=True,
            showgrid=False,
            linecolor='black',
            linewidth=2
        ),
        legend=dict(
            x=0.5,
            y=0.9,
            traceorder='normal',
            font=dict(
                family='Times New Roman',
                size=12,
                color='#000'
            ),
            bordercolor='#000000',
            borderwidth=2
        ), 
        sliders=sliders
    )

    fig = dict(data=data, layout=layout)

    iplot(fig, filename = 'basic-line', config = config)
        
    return cs_outDic_T1values, cs_estimatedT1_values, c_outDic_T1values, c_estimatedT1_values, temperature, sphere;


temperature_correction(np.arange(16,28,2));